In [10]:
import pandas as pd

df = pd.read_csv("co2_annmean_gl.csv", skiprows=37)
print(df.head())

   year    mean   unc
0  1979  336.85  0.10
1  1980  338.91  0.07
2  1981  340.11  0.08
3  1982  340.86  0.03
4  1983  342.53  0.05


In [13]:
import pandas as pd

# 1️⃣ Načtení původního souboru a přeskočení komentářů
df = pd.read_csv("co2_annmean_gl.csv", comment="#")

# 2️⃣ (Volitelné) odstranění prvních N řádků po načtení
df = df.iloc[5:]  # odstraní prvních 5 řádků
df.reset_index(drop=True, inplace=True)

# 3️⃣ Uložení upraveného souboru do nového CSV
df.to_csv("co2_clean.csv", index=False)

In [14]:
import re
from pathlib import Path
import pandas as pd

src = Path("co2_clean.csv")
dst = Path("co2_clean_fixed.csv")

text = src.read_text(encoding="utf-8")

# 0) sjednotit „exotické“ mezery na normální mezeru
text = (text
        .replace("\u00A0", " ")   # NBSP
        .replace("\u202F", " ")   # narrow NBSP
        .replace("\u2009", " ")   # thin space
        .replace("\u2007", " ")   # figure space
)

# 1) vícenásobné tečky (i s mezerami) mezi čísly -> 1 tečka
text = re.sub(r"(?<=\d)(?:\s*\.\s*)+(?=\d)", ".", text)

# 2) mezery před tečkou (uvnitř čísla)
text = re.sub(r"(\d)\s+(?=\.)", r"\1", text)

# 3) mezery za tečkou (uvnitř čísla)
text = re.sub(r"(?<=\d)\.\s+(?=\d)", ".", text)

# 4) mezery kolem oddělovače sloupců
text = re.sub(r"\s*,\s*", ",", text)

dst.write_text(text, encoding="utf-8")

# volitelně: normalizace názvů sloupců
tmp = pd.read_csv(dst)
tmp = tmp.rename(columns={c: c.strip().lower() for c in tmp.columns})
tmp.to_csv(dst, index=False)

# import s typy
df = pd.read_csv(dst, dtype={"year": int, "mean": float, "unc": float})
print(df.dtypes)
print(df.head())

year      int64
mean    float64
unc     float64
dtype: object
   year    mean   unc
0  1984  344.07  0.07
1  1985  345.54  0.07
2  1986  346.97  0.07
3  1987  348.68  0.09
4  1988  351.16  0.06


In [17]:
import re
import pandas as pd
from pathlib import Path

src = Path("co2_annmean_gl.csv")
dst = Path("co2_clean_fixed_safe.csv")

# --- Helpery ---
def normalize_exotic_spaces(s: str) -> str:
    if s is None: return s
    return (s.replace("\u00A0", " ")
              .replace("\u202F", " ")
              .replace("\u2009", " ")
              .replace("\u2007", " "))

def fix_numeric_artifacts(s: str) -> str:
    if s is None: return s
    t = normalize_exotic_spaces(str(s))
    t = re.sub(r"(?<=\d)(?:\s*\.\s*)+(?=\d)", ".", t)   # runy teček -> 1 tečka
    t = re.sub(r"(\d)\s+(?=\.)", r"\1", t)              # mezery před tečkou
    t = re.sub(r"(?<=\d)\.\s+(?=\d)", ".", t)           # mezery za tečkou
    return t.strip()

def safe_cast(series: pd.Series, kind: str) -> pd.Series:
    # Přetypuj jen tam, kde to půjde; jinde nech původní text
    coerced = pd.to_numeric(series, errors="coerce")
    out = series.copy()
    if kind == "int":
        mask = coerced.notna() & (coerced % 1 == 0)
        out.loc[mask] = coerced.loc[mask].astype("int64").astype(str)
    else:
        mask = coerced.notna()
        out.loc[mask] = coerced.loc[mask].astype(float).astype(str)
    return out

# --- Načtení jako čistý text ---
df = pd.read_csv(src, dtype=str, keep_default_na=False)

# Najdi cílové sloupce podle názvu (case-insensitive), ale názvy NEMĚŇ
lower_map = {c.lower().strip(): c for c in df.columns}
targets = {k: lower_map[k] for k in ("year", "mean", "unc") if k in lower_map}

# Oprav jen vybrané sloupce
for k, col in targets.items():
    df[col] = df[col].map(fix_numeric_artifacts)

# Pokus o přetypování JEN tam, kde to jde (jinak zůstane původní řetězec)
if "year" in targets: df[targets["year"]] = safe_cast(df[targets["year"]], "int")
if "mean" in targets: df[targets["mean"]] = safe_cast(df[targets["mean"]], "float")
if "unc"  in targets: df[targets["unc"]]  = safe_cast(df[targets["unc"]],  "float")

# Ulož beze změny struktury/hlaviček
df.to_csv(dst, index=False)
print("Uloženo:", dst)

Uloženo: co2_clean_fixed_safe.csv
